In [7]:
import pandas as pd
import statsmodels.api as sm
df = pd.read_excel(r"C:\Users\sshan\Desktop\DS\Project 4\State export.xlsx", sheet_name = "Main") # load dataset for state-level regression
df = df.iloc[3:].reset_index(drop = True) # optional to ignore certain starting rows 
df["PR"] = df["HRC"]*0.1-df["MPRC"]*0.8-df["PRC"]*0.1 # create a composite price change variable
# define regression function
def regression(y, Xvars):
    X = df[Xvars]
    X = sm.add_constant(X)
    y = df[y]
    model = sm.OLS(y,X).fit(cov_type='HC3')
    return model
Share_model = regression("Haryana % share", ["Haryana share lag","PR","HS diff lag"]) # pass variables and execute regression
cov_matrix = Share_model.cov_params()
print("Share regression:")
print(Share_model.summary())
print("Share parameters:")
print(Share_model.params.to_dict())
print(cov_matrix)

Share regression:
                            OLS Regression Results                            
Dep. Variable:        Haryana % share   R-squared:                       0.940
Model:                            OLS   Adj. R-squared:                  0.915
Method:                 Least Squares   F-statistic:                     13.91
Date:                Sat, 18 Oct 2025   Prob (F-statistic):            0.00246
Time:                        13:59:02   Log-Likelihood:                 26.895
No. Observations:                  11   AIC:                            -45.79
Df Residuals:                       7   BIC:                            -44.20
Df Model:                           3                                         
Covariance Type:                  HC3                                         
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
const               

C:\Users\sshan\anaconda3\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=11
  res = hypotest_fun_out(*samples, **kwds)


In [19]:
import pandas as pd
import statsmodels.api as sm
df = pd.read_excel(r"C:\Users\sshan\Desktop\DS\Project 4\State export.xlsx", sheet_name = "Main")
df = df.iloc[3:].reset_index(drop = True)
df["PR"] = df["MPRC"]*0.7-df["PRC"]*0.1-df["HRC"]*0.2
df["MP diff"] = df["MP rate"]-df["Punjab rate"]
def regression(y, Xvars):
    X = df[Xvars]
    X = sm.add_constant(X)
    y = df[y]
    model = sm.OLS(y,X).fit(cov_type = 'HC3')
    return model
Share_model = regression("MP % share", ["MP share lag","HSC"])
cov_matrix = Share_model.cov_params()
print("Share regression:")
print(Share_model.summary())
print("Share parameters:")
print(Share_model.params.to_dict())
print(cov_matrix)

Share regression:
                            OLS Regression Results                            
Dep. Variable:             MP % share   R-squared:                       0.831
Model:                            OLS   Adj. R-squared:                  0.789
Method:                 Least Squares   F-statistic:                     10.43
Date:                Sun, 19 Oct 2025   Prob (F-statistic):            0.00590
Time:                        23:10:58   Log-Likelihood:                 27.172
No. Observations:                  11   AIC:                            -48.34
Df Residuals:                       8   BIC:                            -47.15
Df Model:                           2                                         
Covariance Type:                  HC3                                         
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
const            0.1629      0

C:\Users\sshan\anaconda3\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=11
  res = hypotest_fun_out(*samples, **kwds)


In [22]:
import pandas as pd
import statsmodels.api as sm
import numpy as np

df = pd.read_excel(r"C:\Users\sshan\Desktop\DS\Project 4\State export.xlsx", sheet_name = "Main")
df = df.iloc[3:].reset_index(drop = True)
df["PR"] = df["HRC"]*0.1-df["MPRC"]*0.8-df["PRC"]*0.1

y_col = "Haryana % share"
x_cols = ["Haryana share lag","PR","HS diff lag"]

y_true = []
y_pred = []

#Leave one out cross validation
for i in range(len(df)):
    train = df.drop(i).reset_index(drop = True)
    test = df.iloc[[i]].reset_index(drop = True)

    x_train = sm.add_constant(train[x_cols], has_constant = "add")
    y_train = train[y_col]

    x_test = sm.add_constant(test[x_cols], has_constant = "add")
    y_test = test[y_col].iloc[0]

    model_i = sm.OLS(y_train, x_train).fit()

    y_hat = float(model_i.predict(x_test))
    y_true.append(y_test)
    y_pred.append(y_hat)

error = np.array(y_true) - np.array(y_pred)
mse = np.mean(error**2)
rmse = np.sqrt(mse)

print(f"LOOCV mse: {mse:.4f}")
print(f"LOOCV rmse: {rmse:.4f}")

result_df = pd.DataFrame({"Actual": y_true, "Pred": y_pred})
print(result_df)

LOOCV mse: 0.0013
LOOCV rmse: 0.0364
      Actual      Pred
0   0.185326  0.171290
1   0.130844  0.196785
2   0.124708  0.101487
3   0.149670  0.185275
4   0.249910  0.230651
5   0.341827  0.328261
6   0.371019  0.316277
7   0.347550  0.410319
8   0.294788  0.276320
9   0.298957  0.282850
10  0.262321  0.272281


C:\Users\sshan\AppData\Local\Temp\ipykernel_14888\285003172.py:28: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  y_hat = float(model_i.predict(x_test))
C:\Users\sshan\AppData\Local\Temp\ipykernel_14888\285003172.py:28: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  y_hat = float(model_i.predict(x_test))
C:\Users\sshan\AppData\Local\Temp\ipykernel_14888\285003172.py:28: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  y_hat = float(model_i.predict(x_test))
C:\Users\sshan\AppData\Local\Temp\ipykernel_14888\285003172.py:28: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  y_hat = float(model_i.predict(x_tes